In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Dict, TypedDict, List

# Define our state
class TranslationState(TypedDict):
    input_text: str
    source_language: str
    target_language: str
    translated_text: str

# Create the translation node using LangChain
def create_translation_node():
    # Define a translation prompt
    translation_prompt = ChatPromptTemplate.from_template(
        """Translate the following text from {source_language} to {target_language}.
        
        Text: {input_text}
        
        Translation:"""
    )
    
    # Create LLM
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    # Create translation chain
    translation_chain = translation_prompt | llm
    
    return translation_chain



def translate_node(state: TranslationState) -> TranslationState:
    # Use the LangChain translation chain
    translation_chain = create_translation_node()
    
    # Run the translation
    result = translation_chain.invoke({
        "input_text": state["input_text"],
        "source_language": state["source_language"],
        "target_language": state["target_language"]
    })
    
    # Extract the translated text from the AIMessage
    translated_text = result.content
    
    # Update state with translated text
    state["translated_text"] = translated_text
    
    return state



# Create the graph
# Add an edge from START to "start"
def create_translation_graph():
    # Initialize the graph
    workflow = StateGraph(TranslationState)
    
    # Add nodes
    workflow.add_node("translate", translate_node)
    workflow.set_entry_point("translate")

    
    # Add edges


    
    # Compile the graph
    return workflow.compile()

trans_app = create_translation_graph()





In [2]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        trans_app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API
        )
    )
)

NameError: name 'trans_app' is not defined

In [ ]:
# Example usage
if __name__ == "__main__":
    # Create the graph
    translation_graph = create_translation_graph()
    
    # Run the graph with an example
    result = translation_graph.invoke({
        "input_text": "Hello, how are you?",
        "source_language": "English",
        "target_language": "Spanish"
    })
    
    print(f"Final result: {result['translated_text']}")